<a href="https://colab.research.google.com/github/noallynoclan/colab/blob/master/210210_taxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import numpy as np
import pandas as pd
from collections import deque

def read_csv(filename):
    columns = {
        'Symbol': 'ticker', 
        'Trade Date': 'date',
        'Purchase Price': 'price',
        'Quantity': 'volume'
        }
    df = pd.read_csv(filename)[columns.keys()].rename(columns=columns)
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
    df = df.sort_values(['date', 'volume'], ascending=[True, False]).reset_index()
    df['tax_year_ru'] = df['date'].dt.year
    df['tax_year_uk'] = df['date'].apply(lambda x: x.year - (0 if x.month > 3 else 1))
    return df

def fifo(df):
    d = {t: deque() for t in df['ticker'].unique()}
    for i, row in df.iterrows():
        date, ticker, volume, price = row[['date', 'ticker', 'volume', 'price']]
        if volume > 0: 
            d[ticker].append([volume, price])
            continue
        fi = []
        v = -volume
        while v:
            if d[ticker][0][0] > v:
                fi.append([v, d[ticker][0][1]])
                d[ticker][0][0] -= v
                v = 0
            elif d[ticker][0][0] <= v:
                v -= d[ticker][0][0]
                fi.append(d[ticker].popleft())
        pprice = np.array(fi).prod(axis=1).sum() / -volume
        df.loc[i, 'pprice'] = pprice
    df['profit'] = (df['pprice'] - df['price']) * df['volume']
    return df

# New Section

In [98]:
df = read_csv('r.csv')
df = fifo(df)
df.groupby('tax_year_uk')['profit'].sum()

tax_year_uk
2019       38.44
2020    42955.97
Name: profit, dtype: float64

In [101]:
df = read_csv('a.csv')
df = fifo(df)
df.groupby('tax_year_ru')['profit'].sum()

tax_year_ru
2018         0.00
2019     22665.32
2020    135793.18
2021     62903.02
Name: profit, dtype: float64

In [53]:
50 + 5 + 28 + 43 + 30 + 24 + 20

200